## Distributed Tensorflow

In [1]:
import tensorflow as tf

x = tf.constant(2)
y1 = x + 300
y2 = x - 66
y = y1 + y2

with tf.Session() as sess:
    result = sess.run(y)
    print(result)

238


In [ ]:
import tensorflow as tf

cluster = tf.train.ClusterSpec({'local' : ['localhost:2222', 'localhost:2223']})
server = tf.train.Server(cluster, job_name='local', task_index=1)

server.start()
server.join()

In [9]:
import tensorflow as tf

cluster = tf.train.ClusterSpec({'local' : ['localhost:2222', 'localhost:2223']})

x = tf.constant(2)

with tf.device('/job:local/task:1'):
    y2 = x - 66
    
with tf.device('/job:local/task:0'):
    y1 = x + 300
    y = y1 + y2
    
with tf.Session('grpc://localhost:2222') as sess:
    result = sess.run(y)
    print(result)

238


## Map and Reduce

In [10]:
def myfunction(x):
    return x + 5

map_result = map(myfunction, [1, 2, 3])

print(list(mabp_result))

[6, 7, 8]


In [11]:
from functools import reduce

def add(a, b):
    return a + b

print(reduce(add, [1, 2, 3]))

6


In [1]:
import numpy as np
import tensorflow as tf

x = tf.placeholder(tf.float32, 100)

mean = tf.reduce_mean(x)

with tf.Session() as sess:
    result = sess.run(mean, feed_dict={x:np.random.random(100)})
    print(result)

0.478785


In [4]:
import numpy as np
import tensorflow as tf

cluster = tf.train.ClusterSpec({'local': ['localhost:2222', 'localhost:2223']})

x = tf.placeholder(tf.float32, 100)

with tf.device('/job:local/task:1'):
    first_batch = tf.slice(x, [0], [50])
    mean1 = tf.reduce_mean(first_batch)
    
with tf.device('/job:local/task:0'):
    second_batch = tf.slice(x, [50], [-1])
    mean2 = tf.reduce_mean(second_batch)
    mean = (mean1 + mean2) / 2
    
with tf.Session('grpc://localhost:2222') as sess:
    result = sess.run(mean, feed_dict={x: np.random.random(100)})
    print(result)

0.460536
